<a href="https://colab.research.google.com/github/hmatny/question_answering/blob/master/Anserini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Environment Setup

#### Download and install maven

In [0]:
!wget http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
!tar -xvzf apache-maven-3.6.1-bin.tar.gz -C /opt

--2019-04-23 01:34:48--  http://mirror.reverse.net/pub/apache/maven/maven-3/3.6.1/binaries/apache-maven-3.6.1-bin.tar.gz
Resolving mirror.reverse.net (mirror.reverse.net)... 208.100.14.200
Connecting to mirror.reverse.net (mirror.reverse.net)|208.100.14.200|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9136463 (8.7M) [application/x-gzip]
Saving to: ‘apache-maven-3.6.1-bin.tar.gz.1’

apache-maven-3.6.1- 100%[===================>]   8.71M  2.65MB/s    in 3.3s    

2019-04-23 01:34:51 (2.65 MB/s) - ‘apache-maven-3.6.1-bin.tar.gz.1’ saved [9136463/9136463]

apache-maven-3.6.1/README.txt
apache-maven-3.6.1/LICENSE
apache-maven-3.6.1/NOTICE
apache-maven-3.6.1/lib/
apache-maven-3.6.1/lib/slf4j-api.license
apache-maven-3.6.1/lib/checker-compat-qual.license
apache-maven-3.6.1/lib/jsr250-api.license
apache-maven-3.6.1/lib/jcl-over-slf4j.license
apache-maven-3.6.1/lib/org.eclipse.sisu.plexus.license
apache-maven-3.6.1/lib/animal-sniffer-annotations.license
apache-maven-

#### Set environment variables

In [0]:
import os
os.environ['PATH'] += ':/usr/lib/jvm/java-8-openjdk-amd64/bin:/opt/apache-maven-3.6.1/bin'
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['M2_HOME'] = "/opt/apache-maven-3.6.0"

# sanity check
!mvn -version

Apache Maven 3.6.1 (d66c9c0b3152b2e69ee9bac180bb8fcc8e6af555; 2019-04-04T19:00:29Z)
Maven home: /opt/apache-maven-3.6.1
Java version: 1.8.0_191, vendor: Oracle Corporation, runtime: /usr/lib/jvm/java-8-openjdk-amd64/jre
Default locale: en_US, platform encoding: UTF-8
OS name: "linux", version: "4.14.79+", arch: "amd64", family: "unix"


#### Clone Anserini git repo

In [0]:
!git clone https://github.com/castorini/Anserini.git
!cd Anserini && mvn clean package appassembler:assemble

Cloning into 'Anserini'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 7062 (delta 25), reused 23 (delta 4), pack-reused 7006
Receiving objects: 100% (7062/7062), 11.16 MiB | 12.37 MiB/s, done.
Resolving deltas: 100% (3693/3693), done.
[INFO] Scanning for projects...
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-compiler-plugin/3.7.0/maven-compiler-plugin-3.7.0.pom (11 kB at 27 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/plugins/maven-plugins/30/maven-plugins-30.pom (10 kB at 318 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/maven/maven-parent/30/maven-parent-30.pom (41 kB at 961 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org/apache/apache/18/apache-18.pom (16 kB at 870 kB/s)
Downloaded from central: https://repo.maven.apache.org/maven2/org

#### Get CAR17 corpus and index data

In [0]:
# !wget http://trec-car.cs.unh.edu/datareleases/v2.0/paragraphCorpus.v2.0.tar.xz
# !tar -xvf paragraphCorpus.v2.0.tar.xz -C /content/Anserini/

In [0]:
# !cd /content/Anserini/ && sh target/appassembler/bin/IndexCollection -collection CarCollection \
# -generator LuceneDocumentGenerator -threads 500000 -input /content/Anserini/paragraphCorpus -index \
# lucene-index.car17a.pos+docvectors+rawdocs -storePositions -storeDocvectors \
# -storeRawDocs

In [0]:
# tar index
# !cd /content/Anserini/ && tar czf lucene-index.car17-large.tar.gz lucene-index.car17a.pos+docvectors+rawdocs

In [0]:
# save index to gs bucket
# !gsutil cp /content/Anserini/lucene-index.car17-large.tar.gz gs://cs378_bert/lucene-index-large.tar.gz

#### Download the Lucene index

In [0]:
!gsutil cp gs://cs378_bert/lucene-index-large.tar.gz /tmp/lucene-index-large.tar.gz
!cd /tmp && tar -xvzf lucene-index-large.tar.gz -C /content/Anserini/

Copying gs://cs378_bert/lucene-index-large.tar.gz...
- [1 files][ 14.1 GiB/ 14.1 GiB]   50.0 MiB/s                                   
Operation completed over 1 objects/14.1 GiB.                                     
lucene-index.car17a.pos+docvectors+rawdocs/
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tip
lucene-index.car17a.pos+docvectors+rawdocs/_2.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_2.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.tim
lucene-index.car17a.pos+docvectors+rawdocs/_1_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_2_Lucene50_0.doc
lucene-index.car17a.pos+docvectors+rawdocs/_3.nvd
lucene-index.car17a.pos+docvectors+rawdocs/_0.fnm
lucene-index.car17a.pos+docvectors+rawdocs/_0_Lucene70_0.dvd
lucene-index.car17a.pos+docvectors+rawdocs/_1.tvx
lucene-index.car17a.pos+docvectors+rawdocs/_0.si
lucene-index.car17a.pos+docvectors+rawdocs/_1.fdx
lucene-index.car17a.pos+d

#### Install dependencies

In [0]:
!pip3 install pyjnius

!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py

  Stored in directory: /root/.cache/pip/wheels/c1/2d/85/9884050da2f10b9f72b029f34bedef0993c339437aa956906f
Successfully built pyjnius
--2019-04-23 01:45:15--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.004s  

2019-04-23 01:45:15 (9.57 MB/s) - ‘modeling.py’ saved [37922/37922]

--2019-04-23 01:45:17--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|1

### Using Anserini 

In [0]:
# Only run this once
import jnius_config
jnius_config.set_classpath('Anserini/target/anserini-0.4.1-SNAPSHOT-fatjar.jar')

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
questions = ['What is Google?', 'Who is Michael Jordan?']

In [0]:
import run_squad
from nltk.tokenize import sent_tokenize
from jnius import autoclass
JString = autoclass('java.lang.String')
JSearcher = autoclass('io.anserini.search.SimpleSearcher')

searcher = JSearcher(JString('Anserini/lucene-index.car17a.pos+docvectors+rawdocs'))

def get_context(question,filename):
  hits = searcher.search(JString(question),10)
  context_dict = {}
  data = {}
  data["title"] = filename
  counter = 0
  p_list = []
  
  score_map = {}
  for hit in hits:
    p_context = {}
    p_context["context"] = hit.content
    p_context["qas"] = [{"answers": [], "question":question, "id":counter}]
    p_list.append(p_context)
     
    
    counter +=1 
    score_map[counter] = hit.score

  data["paragraphs"] = p_list
  
  root_dict = {}
  root_dict["data"] = [data]
  
  return root_dict,score_map



def calc_best_cand(root_dict, ):
  
# examples = []
# for question in questions:
#   hits = searcher.search(JString(question),20)
#   for hit in hits:
#     example = run_squad.SquadExample("0", question, sent_tokenize(hit.content))
#     examples.append(example)
    


# BERTsirini - setting up env


Download Required material

In [0]:
#these download all the reqiured BERT material, not sure if they are all required yet
!mkdir /content/BERT/

!gsutil cp -r gs://cs378_bert/model_repo  /content/BERT/
# !gsutil cp -r gs://cs378_bert/bert-tfhub  /content/BERT/  
# !gsutil cp -r gs://cs378_bert/bert-checkpoints  /content/BERT/
 

Copying gs://cs378_bert/model_repo/outputs/checkpoint...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555284422.f53183836553...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555358512.26db6926706b...
| [3 files][ 20.6 MiB/ 20.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555359174.26db6926706b...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555359398.26db6926706b...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555516188.fa30e7de8aaf...
Copying gs://cs378_bert/model_repo/outputs/events.out.tfevents.1555707495.dd846ac09b98...
Copying gs://cs378_bert/model_repo/outputs/graph.pbtxt...
Copying gs:/

TPU settings

In [0]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import zipfile
from matplotlib import pyplot as plt
%matplotlib inline
import sys
import datetime
import json
import tensorflow as tf

In [0]:
#NOT SURE IF WE NEED THIS RIGHT NOW

# BUCKET = 'cs378_bert' #@param {type:"string"}
# assert BUCKET, 'Must specify an existing GCS bucket name'
# REPO ='model_repo'
# BUCKET_OUTPUT_DIR = 'gs://{}/{}/outputs'.format(BUCKET,REPO)
# BUCKET_PREFIX = "gs://cs378_bert"
# tf.gfile.MakeDirs(BUCKET_OUTPUT_DIR)
# print('***** Model output directory: {} *****'.format(BUCKET_OUTPUT_DIR))
 
# !wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 
# !wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py
!wget https://raw.githubusercontent.com/google-research/bert/master/run_squad.py 

--2019-04-23 01:47:05--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘optimization.py.1’

optimization.py.1   100%[===================>]   6.11K  --.-KB/s    in 0s      

2019-04-23 01:47:05 (88.3 MB/s) - ‘optimization.py.1’ saved [6258/6258]

--2019-04-23 01:47:06--  https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12257 (12K) [text/plain]
Saving t

In [0]:
import json
# grab context from anserini
# format context+question in json for bert to read
QUESTION = 'How many championships do the spurs have?' #@param {type:"string"}
FILENAME = 'spurs' #@param {type:"string"}
context_dict,scores = get_context(QUESTION,FILENAME)
FILENAME = FILENAME + ".json"
text_file = open(FILENAME, "w")
text_file.write(json.dumps(context_dict))
# pass input file to bert 
### TYPE FILENAME INTO predict-file= 
!python run_squad.py \
  --vocab_file='BERT/model_repo/uncased_L-12_H-768_A-12/vocab.txt' \
  --bert_config_file='BERT/model_repo/uncased_L-12_H-768_A-12/bert_config.json' \
  --init_checkpoint='BERT/model_repo/outputs/' \
  --do_predict=True \
  --predict_file=$FILENAME \
  --train_batch_size=1 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir='./'
bert_pred = open("nbest_predictions.json")
bert_pred = json.load(bert_pred)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f141c548710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num

In [0]:
# !cat predictions.json
# !cat nbest_predictions.json
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)
def calc_score(a_score, b_score,u=0.5):
  return (1-u)*a_score + u*b_score

def normalize(d, target=1.0):
   raw = sum(d.values())
   factor = target/raw
   return {key:value*factor for key,value in d.items()}

def find_best_ans(bert_pred,a_scores,u=0.5):
  u = u
  a_scores = normalize(a_scores,1) #normalize values in anserini scores

  best = 0
  max_score = -1
  for key in range(0,len(bert_pred.keys())-1):
    ans_ind = key+1 #anserini keys indexed at 1
    score_temp = calc_score(a_scores[ans_ind],bert_pred[str(key)][0]["probability"],u)
    if score_temp > max_score:
        max_score = score_temp
        best_key = key
  return best_key, max_score

def print_results(b_key,m_score):
  print("~~~~~~~~~~~~~~~~~~~~")
  print("Confidence score: " + str(m_score))
  print("Answer: " + bert_pred[str(b_key)][0]["text"])
  pp.pprint((context_dict["data"][0]["paragraphs"][b_key]["context"]))
  print("~~~~~~~~~~~~~~~~~~~~")
  


In [0]:
#open nbest_predictions
b_key,m_score = find_best_ans(bert_pred, norm_scores, 0.5)
print_results(b_key,m_score)

~~~~~~~~~~~~~~~~~~~~
Confidence score: 0.5530817039530038
Answer: five
('The rivalry between the San Antonio Spurs and the Dallas Mavericks features '
 'two teams with Dallas roots. The Spurs began their life in the ABA as the '
 'Dallas Chaparrals and did not move to San Antonio until 1973. On October 11, '
 '1980, the Mavs made their NBA debut by defeating the Spurs 103–92. In the '
 'playoffs the Spurs defeated the Mavericks in 2001, 2003, 2010, and 2014; '
 'while the Mavericks defeated the Spurs in 2006 and 2009. The Spurs have won '
 'five championships and six conference titles, while the Mavericks have won '
 'one championship and two conference titles. The Spurs have won 18 division '
 'titles, while the Mavericks have won 3. The Mavericks have three 60-win '
 'seasons, while the Spurs have five.')
~~~~~~~~~~~~~~~~~~~~
